## Preprocessing

In the previous notebook we extracted the time series for different ROIs. However, we are now going to take a step back from that and preprocess our data before we can move on with the analysis.


A key point in the analysis of brain image is to have a solid preprocessed image. The preprocessing inclues all the steps that are taken to improve our data and prepare it for statistical analysis. Usual preprocessing steps include: 
 * Corrections for head movement during scanning 
 * Slice corrections
 * Detection of artifacts during scanning
 * Normalisation of structural image into MNI space
 * Filtering

We are lucky that a few of this steps have been already performed on the dataset we are using. In this notebook we are going to perform a few important preprocessing steps. To do this, we will use a library called *nipype*, so make sure it is installed in your enviroment.

*Nipype* is a really nice library that allows you to use different functions from different existing neuroimaging softwares.


As usual, first we need to load the required Python libraries. Again, make sure you have them all installed in your virtual enviroment.

In [ ]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
%matplotlib inline  
from IPython.display import Image

from nipype.interfaces.fsl import GLM, MeanImage, BinaryMaths, FSLCommand, TemporalFilter
from nipype.interfaces.utility import IdentityInterface
import nipype.pipeline.engine as pe
from nipype import Node, Workflow
from nipype.interfaces.io import DataSink, DataGrabber

**Note**: If you are having problems loading nipype you shoul do the following:
- install ipykernel on your virtual enviroment by doing:
`conda install ipykernel`
- run in the terminal (change my-virtualenv-name):
`python -m ipykernel install --user --name=my-virtualenv-name`
- Then inside your notebook click on `Kernel>Change Kernel>my-virtualenv-name`

If you are not having problems than just ignore this.


Then set up the paths to our data and important variables. You should change them accordingly, so that they match your paths.

In [ ]:
# Settings
base_path_data = os.path.join(os.path.sep, 'group', 'dynamics', 'HCP')
data_out_dir = os.path.join(os.path.sep, 'home', 'jdafflon', 'code', 'msc_project', 'patrycja', 'data_out')


LR_or_RL = 'rfMRI_REST2_LR'
subjects_list = ['100307']
FSLCommand.set_default_output_type('NIFTI_GZ')


segmentation_image_path = os.path.join( os.path.sep, 'group', 'dynamics', 'scz_dynamics', 'ucla-la5', 'data_in', 'voi_extraction', 'seg_aparc_82roi_2mm.nii.gz')
lookuptable_file_path = os.path.join(os.path.sep, 'group', 'dynamics', 'scz_dynamics', 'ucla-la5', 'data_in', 'voi_extraction','LookupTable')
non_preprocessed_image_path = os.path.join(base_path_data, subjects_list[0], 'MNINonLinear', 'Results', LR_or_RL, 'rfMRI_REST2_LR.nii.gz')

We will now do some corrections for motion on the dataset. Subject head movement, during the experiment is a major source of artefact in fMRI data. Changes in pixel intensity at the edges of the brain, upon even slight movement, can be far greater than the BOLD activation response. It is common therefore in fMRI data analysis to perform some correction which reduces this effect.

Luckly, the motion parameters have already been calculated for us, so all we need to do is load them. And check the shape, for good practice.

In [ ]:
# Load motion regressors.


## Nipype

Now we will start using nipype. There are different ways of using nipype, here we will create a workflow where each step of our preprocessing pipeline will be composed by a node. Therefore we will need to create the basics for your Nodes in nipype.

If you want to get a better insight how a piple is create in nipype you can take a look [here]( http://miykael.github.io/nipype-beginner-s-guide/firstSteps.html). But I will save you the work of creating a pipeline and do everything here.


### Temporal Filtering

Before we go into more details of how to run the preprocessing pipeline in nipype. Let's take a look into the temporal filtering, which is an important step in the preprocessing. 

Temporal Filtering allow us to improve the signal-to-noise ratio of our data. It not only removes high frequency fluctuations but also scanner drifts from the data.

There are mainly three types of filters: **high-pass filters**, **low-pass filters** and **band-pass filters**.
While high-pass filtering will remove low frequency variations in the data, the low-pass filter will remove high frequency in the data. The band-pass filter will remove the frequecies that are outside the specified range.

For this preprocessing we wil perform a band-pass filter.
To do this we will need to find the TR of our data. The TR corresponds to the repetition time of our MRI sequence and can be read up from the data.

**Question:** Find out the TR of the current dataset. Hint: Try to call the function `fslinfo` on your resting state image or alternatively you can look up the documentation on the HCP website (where we have downloaded the data).

### Band-pass filter

Spontaneous activity have been observed in the resting state in a low frequency range of [0.01–0.1] Hz. Therefore, we will perform a banpass filter to filter only the frequencies that in the range that we are interested in. Take a look at this [paper](http://www.sciencedirect.com/science/article/pii/S1053811916307881). It will give you an introduction to dynamic Functional Connectivity which we will use in the next notebook and in idea of what are those spontaneous fluctuations.

Let's get back to our analysis. Let's say we want to create a band-pass filter between 0.04 Hz and 0.1. How can we do it?

This example assumes that the TR is 2, which is **not** the case for this dataset but using the TR information you got from your data set you should be able to calcultate it.

* Lower band of the banp-pass filter:

low_pass = $\frac{\frac{1}{TR}}{filter in Hz} = \frac{\frac{1}{2s}}{0.04 1/s} = 12.5 $
However, as the fsl temporal filter takes the sigma HWHM and not the FWHM, we will need to divide this value by 2.
low_pass_filter = $\frac{12.5}{2} = 6.25$

* Higher band of the band-pass filter:
A similar calculation is one using the *high pass filter* just change the 0.04 Hz for 0.1 Hz.

In [ ]:
# Find TR of the dataset
# TR = ??? (in seconds)

lowpass_hz = 0.04
highpass_hz = 0.1

lowpass_sigma = round((1/TR)/(lowpass_hz * 2),2)
highpass_sigma = round((1/TR)/(highpass_hz * 2),2)

print('low-pass sigma:', lowpass_sigma)
print('high-pass sigma:', highpass_sigma)

Because we might want to change this boundary in the future we are going to define a function that takes a list of frequencies as input for the preprocessing.

In [ ]:
def preprocessing_pipeline(lowpass_sigma_list ,highpass_sigma_list):
    # Create input node. This is where you are specify where you data comes from.
    infosource = Node(interface=IdentityInterface(fields=['subject_id']), name='InfoSource')
    infosource.iterables = ('subject_id', subjects_list)
    datasource = Node(interface=DataGrabber(infields=['subject_id'],
                 outfields=['epi']), name='datasource')
    datasource.inputs.base_directory = base_path_data
    datasource.inputs.template = '*'
    datasource.inputs.sort_filelist = True
    datasource.inputs.field_template = dict(epi=os.path.join('%s', 'MNINonLinear', 'Results', LR_or_RL, 'rfMRI_REST2_LR.nii.gz'))    

    # Create data sink. This is where your data is going to be saved.
    data_sink = Node(DataSink(), name='DataSink')
    data_sink.inputs.base_directory = os.path.join(data_out_dir, 'preprocessing_out')
    substitutions = [('_subject_id', '')]
    
    # NODE 1:
    # We will use a General Linear Model (GLM) to regress activity related to movement.
    # To do this we will use the motion parametrs we have from our dataset.
    glm = Node(GLM(), name='GLM')
    glm.inputs.design = motion_parameter_file
    glm.inputs.demean = True
    glm.inputs.out_res_name = 'residuals.nii.gz' 
    glm.inputs.out_file = 'glm_betas.nii.gz'

    # NODE 2:
    # This node performs the temporal filtering.
    tmp_filt = Node(TemporalFilter(), name='TemporalFilter')
    tmp_filt.iterables = [('lowpass_sigma', lowpass_sigma_list), 
                          ('highpass_sigma', highpass_sigma_list)]

    # Node 3:
    # Get mean image. Thi is only needed to get a pretty picture at the end
    mean_img = Node(MeanImage(), name='MeanImage')

    # Node 4:
    # Add the mean image to get a pretty picture
    add_mean = Node(BinaryMaths(), name='AddMean')
    add_mean.inputs.operation='add'
    add_mean.inputs.terminal_output='file'

    # Now that we have defined all nodes we will have to combine them. 
    preproc = Workflow(name='preprocessing')
    preproc.base_dir = data_out_dir
    preproc.connect([
        (infosource, datasource, [('subject_id', 'subject_id'  )] ),
        (datasource, glm,        [('epi'       , 'in_file'     )] ),
        (glm,        tmp_filt,   [('out_res'   , 'in_file'     )] ),
        (tmp_filt,  data_sink,   [('out_file'  , 'tmp_filt'    )] ),
        (tmp_filt,  add_mean,    [('out_file'  , 'in_file'     )] ),
        (datasource, mean_img,   [('epi'       , 'in_file'     )] ),
        (mean_img,   add_mean,   [('out_file'  , 'operand_file')] ),
        (add_mean,   data_sink,  [('out_file'  , 'meaned_image' )] ),
    ])
    preproc.write_graph(os.path.join(data_out_dir, 'workflow_graph'))
    preproc.run()


We can now call our pre-processing function with the correct input

In [ ]:
highpass_sigma_list = [highpass_sigma]
lowpass_sigma_list = [lowpass_sigma]

preprocessing_pipeline(lowpass_sigma_list,highpass_sigma_list)

Plot below the graph that sumarise our current preprocessing pipeline.

In [ ]:
# Plot png image


**Question:** Where is the temporal filtered and demeaned image stored, and where is the image where the mean is added stored? Write down the path to the filtered image and name your variable as `path_to_processed_image`. We are going to need this below.

## Extract ROIs from the preprocessed data

**Question:** Use the code from the 1st python notebook to extract the ROIs for the preprocessed data. The idea is the same as from the 1st noteboook. You are supposed to iterate over the regions, find their intensities and then extract the time series from the resting state image. 

However, this time this code will be englobe in a function. So you will need to transform the code you have written into a function. You should pass the image of interest, the lookuptable and the segmentation image as arguments.  The function should return the average time series.

In [ ]:
#  Define here the function

Now that we have defined the function we can call it in order to extract the ROIs for both the preprocessed and not preprocessed image. 

Load the preprocessed, the non-preprocessed and the segmentation image using nibabel before you pass them to the function. Remember to also load the lookuptable.

In [ ]:
# Load images

In [ ]:
# Call the function you defined twice. Once you will pass the preprocessed image and for the other the non preprocessed
#image


In our analysis. We are concerned about how the correlation between the different ROIs change over time.

**Question:** Visualise the correlation matrix for the preprocessed and not preprocessed image using `np.corrcoef`. Do you notice any difference?

In [ ]:
# plot 2 images side by side
